In [1]:
import numpy as np
from scipy.stats import norm, ncx2
np.random.seed(1234)

Problem 1

In [2]:
r0 = 0.05
sigma = 0.1
kappa = 0.82
r_bar = 0.05
FV = 1000
dt = 1/360.
simulations = 5000

In [19]:
# (a)

T_zcb = 0.5
step_zcb = int(T_zcb * 360)
r_mat = np.zeros((simulations, step_zcb + 1))
r_mat[:,0] = r0

for i in range(1, step_zcb + 1):
    brownian = np.random.standard_normal(simulations)
    r_mat[:,i] = r_mat[:, i-1] + kappa * (r_bar - r_mat[:,i-1]) * dt + sigma * brownian * np.sqrt(dt)
    
R = dt * np.sum(r_mat[:,1:], axis = 1)
zcb = np.mean(np.exp(-R)) * FV
print("1a: " + str(zcb))

1a: 975.578711353


In [26]:
# (b)

T_cp = np.arange(0.5, 4 + 0.5, 0.5)
coupon = 30.
step_cp = [int(360 * i) for i in T_cp]
periods = int(2*T_cp[-1])
C = np.repeat(coupon, periods)
C[-1] += FV  
r_mat_cp = np.zeros((simulations, step_cp[-1] + 1))
r_mat_cp[:,0] = r0

for i in range(1, step_cp[-1] + 1):
    brownian = np.random.standard_normal(simulations)
    r_mat_cp[:,i] = r_mat_cp[:, i-1] + kappa * (r_bar - r_mat_cp[:,i-1]) * dt + sigma * brownian * np.sqrt(dt)
  
R_cp = [dt * np.sum(r_mat_cp[:,1:i+1], axis = 1) for i in step_cp]
zcb_list = [np.mean(np.exp(-R_cp[i])) * C[i] for i in range(periods)] 
cpbond = sum(zcb_list)
print("1b: " + str(cpbond))

1b: 1047.0476765


In [20]:
# (c)

T_call = 3/12.
K_call = 980. 
expiration = int(360*T_call)

def B_V(T_opt, T_bond):
    B = 1/kappa * (1- np.exp(-kappa * (T_bond - T_opt)))
    return B

def A_V(T_opt, T_bond):
    A = np.exp((r_bar - 0.5 * sigma**2/kappa**2) * (B_V(T_opt, T_bond) - (T_bond - T_opt)) - sigma**2 * B_V(T_opt, T_bond)**2/(4*kappa))
    return A

prices_zcb = A_V(T_call, T_zcb) * np.exp(-B_V(T_call, T_zcb) * r_mat[:,expiration]) * 1000
R_zcb = dt * np.sum(r_mat[:,1:expiration + 1], axis = 1)
payoff_zcb = np.maximum(prices_zcb - K_call, 0)
EC_zcb = np.dot(np.exp(-R_zcb), payoff_zcb)/simulations
print("1c: " + str(EC_zcb))

1c: 8.86731060227


In [28]:
#(d)
expiration = int(360*T_call)

def ZCB(t, S, r0, FV, M = 1000):
    step = int((S - t) * 360)
    r_mat = np.zeros((M, step + 1))
    r_mat[:,0] = r0
    
    for i in range(1, step + 1):
        brownian = np.random.standard_normal(M)
        r_mat[:,i] = r_mat[:, i-1] + kappa * (r_bar - r_mat[:,i-1]) * dt + sigma * brownian * np.sqrt(dt)
        
    R = dt * np.sum(r_mat, axis = 1)
    zcb = np.mean(np.exp(-R)) * FV
    return zcb

r_start = r_mat_cp[:, expiration]
prices_part_im = []

for j in range(8):
    zcb_next = np.array([ZCB(T_call, T_cp[j], r_start[i], C[j]) for i in range(simulations)])
    prices_part_im.append(zcb_next)

prices_part_im = np.array(prices_part_im)
prices_cp_im = np.sum(prices_part_im, axis = 0)
payoff_cp_im = np.maximum(prices_cp_im - K_call, 0)
R_cp = dt * np.sum(r_mat_cp[:,1:expiration+1], axis = 1)
EC_cp_im = np.dot(np.exp(-R_cp), payoff_cp_im)/simulations
print("1d: " + str(EC_cp_im))

1d: 82.8790358168


In [27]:
# (e)
# explict formula 
prices_part_ex = np.array([A_V(T_call, T_cp[i]) * np.exp(-B_V(T_call, T_cp[i]) * r_mat_cp[:, expiration]) * C[i] for i in range(periods)])
prices_cp_ex = np.sum(prices_part_ex, axis = 0)
payoff_cp_ex = np.maximum(prices_cp_ex - K_call, 0)
R_cp = dt * np.sum(r_mat_cp[:,1:expiration+1], axis = 1)
EC_cp_ex = np.dot(np.exp(-R_cp), payoff_cp_ex)/simulations
print("1e: " + str(EC_cp_ex))


1e: 82.9288822969


Comment: The results from (d) and (e) are quite close. Using explicit formula for the underlying bond price will yield better performance. 


Problem 2

a. Monte-Carlo simulation

In [5]:
# (a)
sigma_cir = 0.12
kappa_cir = 0.92
r_bar_cir = 0.055
step_cir = int(0.5 * 360)
K_call = 980.

h1 = np.sqrt(kappa_cir**2 + 2*sigma_cir**2)
h2 = (kappa_cir + h1)/2.
h3 = 2 * kappa_cir * r_bar_cir/sigma_cir**2

def B_CIR(t, T):
    B = (np.exp(h1 * (T - t)) - 1)/(h2 * (np.exp(h1 * (T - t)) - 1) + h1)
    return B

def A_CIR(t, T):
    A = np.power(h1 * np.exp(h2 * (T - t))/(h2 * (np.exp(h1 * (T - t)) - 1) + h1),h3)
    return A


r_mat_cir = np.zeros((simulations, step_cir + 1))
r_mat_cir[:,0] = r0
for i in range(1, step_cir + 1):
    brownian = np.random.standard_normal(simulations)
    r_mat_cir[:,i] = r_mat_cir[:, i-1] + kappa_cir * (r_bar_cir - r_mat_cir[:,i-1]) * dt + sigma_cir * brownian * np.sqrt(dt * r_mat_cir[:, i-1])


In [6]:
r_start_cir = r_mat_cir[:,-1]
newstep = int(360*0.5)
newpath = 1000
prices_cir_im = []

for i in range(simulations):
    r = np.zeros((newpath, newstep+1))
    r[:,0] = r_start_cir[i]
    
    for j in range(1, newstep+1):
        brownian = np.random.standard_normal(newpath)
        r[:,j] = r[:, j-1] + kappa_cir * (r_bar_cir - r[:,j-1]) * dt + sigma_cir * brownian * np.sqrt(dt * r[:,j-1])
    
    R = np.sum(r[:,1:], axis = 1) * dt
    zcb_next = np.mean(np.exp(-R)) * 1000
    prices_cir_im.append(zcb_next)


R_cir = dt * np.sum(r_mat_cir[:,1:], axis = 1)
payoff_cir_im = np.maximum(np.array(prices_cir_im) - K_call, 0)
EC_cir_im = np.dot(np.exp(-R_cir), payoff_cir_im)/simulations
print("2a: "+ str(EC_cir_im))

2a: 0.387307160898


b. Implict finite difference method

In [49]:
M = int(0.5 * 360) # time grid
N = 100 # space grid
deltar = 0.001
r_seq = np.linspace(N*deltar, 0, N+1)
j = np.arange(N-1, 0, -1, dtype = np.float)

pu = 0.5 * dt *(-sigma_cir**2 * j/deltar - kappa_cir * r_bar_cir/deltar + kappa_cir * j)
pm = 1 + sigma_cir**2 *j * dt/deltar + j * deltar * dt
pd = 0.5 * dt *(-sigma_cir**2 * j/deltar + kappa_cir * r_bar_cir/deltar - kappa_cir * j)

A = np.zeros((N+1, N+1))
for i in range(1, N):
    A[i,i-1:i+2] = pu[i-1], pm[i-1], pd[i-1]
A[0,:2] = [1,-1]
A[-1,N-1:] = [1,-1]
 
F = np.zeros((N+1, M+1))

prices_end = np.array(A_CIR(0.5, 1.) * np.exp(-B_CIR(0.5, 1.) * r_seq) * 1000)
F[:,-1] = np.maximum(prices_end - K_call, 0) 

for i in range(M-1, -1, -1):
    B = F[:,i+1]
    B[0] = 0 
    B[-1] = 1000 * (A_CIR(0.5, 1.) * np.exp(-B_CIR(0.5, 1.) * r_seq[-1]) - A_CIR(0.5, 1.) * np.exp(-B_CIR(0.5, 1.) * r_seq[-2]))
    F[:,i] = np.linalg.inv(A) @ B

optprices = F[:,0]
EC_2b = np.interp(r0, r_seq[::-1], optprices[::-1])
print("2b: " + str(EC_2b))

2b: 0.39292111966639653


c. Explicit formula

In [50]:
theta = np.sqrt(kappa_cir**2 + 2 * sigma_cir**2)
phi = 2 * theta /sigma_cir**2/(np.exp(theta * 0.5) - 1)
psi = (kappa_cir + theta)/sigma_cir**2
rstar = np.log(A_CIR(0.5, 1.)/K_call*1000.) /B_CIR(0.5, 1.)
P_S_cir = A_CIR(0., 1.) * np.exp(-B_CIR(0., 1.) * r0) * FV
P_T_cir = A_CIR(0., 0.5) * np.exp(-B_CIR(0., 0.5) * r0) * FV
EC_2c = P_S_cir * ncx2.cdf(2*rstar*(phi + psi + B_CIR(0.5, 1.)), 4*kappa_cir*r_bar_cir/sigma_cir**2, 
                         (2 * phi**2 * r0 * np.exp(theta*0.5))/(phi + psi + B_CIR(0.5, 1.))) - K_call/1000 * P_T_cir * ncx2.cdf(2*rstar*(phi + psi), 
                         4*kappa_cir*r_bar_cir/sigma_cir**2,  (2 * phi**2 * r0 * np.exp(theta*0.5))/(phi + psi))

print("2c: " + str(EC_2c))

2c: 0.394057532617


Comment: Comparing with Monte-Carlo simulation, the Finite Difference Method yields more precise result (closer to result of the explicit formula). One possible way to enhance the presicion of Monte-Carlo Method is to increase the paths of simulations.

Problem 3

Monte-carlo simulation

In [7]:
# (a)
phi_3 = 0.03
r0_3 = 0.03
rho = 0.7
a = 0.1
b = 0.3
sigma_3 = 0.03
eta = 0.08
K_put = 950
T_put = 0.5
S = 1 
step_put = int(T_put * 360)

def ZCB_G2(t, T, xt, yt, FV):
    tau = T - t
          
    v = sigma_3**2/a**2 * (tau + 2/a * np.exp(-a * tau) - 0.5/a * np.exp(-2*a*tau) - 1.5/a) + \
    eta**2/b**2 * (tau + 2/b * np.exp(-b * tau) - 0.5/b * np.exp(-2*b*tau) - 1.5/b) + \
    2 * rho * sigma_3 * eta /(a*b) * (tau + (np.exp(-a*tau) -1)/a + (np.exp(-b*tau) -1)/b - (np.exp(-(a + b) * tau) -1)/(a+b))
        
    disc = np.exp(- phi_3 * tau - xt * (1 - np.exp(-a * tau))/a - yt * (1 - np.exp(-b * tau))/b + v/2.)
    
    return disc * FV 


In [10]:
r_mat3 = np.zeros((simulations, step_put + 1))
x_mat = np.zeros_like(r_mat3)
y_mat = np.zeros_like(r_mat3)
r_mat3[:,0] = r0_3
for i in range(1, step_put + 1):
    brownian = np.random.standard_normal(2 * simulations)
    # generate Bivariate-Normally distributed random variables with given correlations
    z1 = brownian[:simulations]
    z2 = rho * z1 + np.sqrt(1 - rho**2) * brownian[simulations:]
    x_mat[:,i] = x_mat[:,i-1] - a * x_mat[:,i-1] * dt + sigma_3 * np.sqrt(dt) * z1
    y_mat[:,i] = y_mat[:,i-1] - b * y_mat[:,i-1] * dt + eta * np.sqrt(dt) * z2
    r_mat3[:,i] = x_mat[:,i] + y_mat[:,i] + phi_3
    
x_end_put = x_mat[:,step_put]
y_end_put = y_mat[:,step_put]


In [11]:
r_start_g2 = r_mat3[:,-1]
prices_g2_im = []

for i in range(simulations):
    r = np.zeros((newpath, newstep+1))
    x = np.zeros_like(r)
    y = np.zeros_like(r)
    r[:,0] = r_start_g2[i]
    x[:,0] = x_end_put[i]
    y[:,0] = y_end_put[i]
    
    for j in range(1, newstep+1):
        brownian = np.random.standard_normal(2 * newpath)
        # generate Bivariate-Normally distributed random variables with given correlations
        z1 = brownian[:newpath]
        z2 = rho * z1 + np.sqrt(1 - rho**2) * brownian[newpath:]
        x[:,j] = x[:,j-1] - a * x[:,j-1] * dt + sigma_3 * np.sqrt(dt) * z1
        y[:,j] = y[:,j-1] - b * y[:,j-1] * dt + eta * np.sqrt(dt) * z2
        r[:,j] = x[:,j] + y[:,j] + phi_3
    
    R = np.sum(r[:,1:], axis = 1) * dt
    zcb_next = np.mean(np.exp(-R)) * 1000
    prices_g2_im.append(zcb_next)


R_g2 = dt * np.sum(r_mat3[:,1:], axis = 1)
payoff_g2_im = np.maximum(float(K_put) - np.array(prices_g2_im), 0)
EC_g2_im = np.dot(np.exp(-R_g2), payoff_g2_im)/simulations
print("3a: " + str(EC_g2_im))

3a: 1.84057039645


Explict formula

In [34]:
SIGMA = np.sqrt(sigma_3**2/(2 * a**3) * pow(1 - np.exp(-a * (S - T_put)), 2) * (1 - np.exp(-2* a * T_put)) + 
                eta**2/(2 * b**3) * pow(1 - np.exp(-b * (S - T_put)), 2) * (1 - np.exp(-2 * b * T_put)) + 
                2*rho*sigma_3*eta/(a*b*(a+b)) * (1 - np.exp(-a*(S -T_put))) * (1 - np.exp(-b*(S -T_put))) * (1 - np.exp(-(a+b)*T_put)))

P_S_g2 = ZCB_G2(0, S, 0, 0, 1000)
P_T_g2 = ZCB_G2(0, T_put, 0, 0, 1000)

EP_formula = - P_S_g2 * norm.cdf(np.log(K_put/1000 * P_T_g2/P_S_g2)/SIGMA - SIGMA/2.) + P_T_g2 * K_put/1000. * norm.cdf(np.log(K_put/1000 * P_T_g2/P_S_g2)/SIGMA + SIGMA/2.) 
print("3b: " + str(EP_formula))

3b: 1.86095987496


Comment: The results from Monte-Carlo simulation vary around 1.86 as we input different seeds. One possible way to enhance the presicion of Monte-Carlo Method is to increase the paths of simulations.